<a href="https://colab.research.google.com/github/veiro/tesis-imputacion-datos/blob/main/codigo/imputadores/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEER
1. Cambiar en enterno a GPU y la variable cuda= True
2. Para ejecutar todo el dataset, MODO_DESARROLLO=False, USAR_GOOGLE_DRIVE =  False y ejecutar un docker local de colab por que demora mas de 12 hrs.
3. tomado de https://zenodo.org/records/10016161

## 1. Requirements

In [ ]:
USAR_GOOGLE_DRIVE =  True
MODO_DESARROLLO = False

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from sklearn.neighbors import KNeighborsRegressor

In [ ]:
if (USAR_GOOGLE_DRIVE):
  from google.colab import drive
  drive.mount('/content/gdrive')
  PATH_DATA_PROCESADA = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/data/data-procesada"
  PATH_UTILS= "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/utils"
else:
  PATH_DATA_PROCESADA = "C:\\Users\\aveiro\\Desktop\\Tesis\\data\\data-procesada"
  PATH_UTILS = "C:\\Users\\aveiro\\Desktop\\Tesis\\Codigo\\utils\\"

sys.path.append(PATH_UTILS)

import utils as utils
import importlib
importlib.reload(utils)

if (30 ==  utils.version()):
  print("version correcta")
else:
  raise Exception("Version de Util vieja")

Mounted at /content/gdrive
version correcta


In [ ]:
!python3 -m pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.5
cufflinks==0.17.3
cupy-cuda12x==12.2.0

## 3. Prepare Data

## Armo juego de datos entramiento y test


In [ ]:
df_train_encoded, df_train_encoded_missing, mask_train_open = utils.obtenerDatosTrain(MODO_DESARROLLO, PATH_DATA_PROCESADA)
df_test_encoded, df_test_encoded_missing, mask_test_open = utils.obtenerDatosTest(MODO_DESARROLLO, PATH_DATA_PROCESADA)

df_test_encoded = pd.DataFrame(df_test_encoded, columns = df_train_encoded.columns)
df_test_encoded_missing = pd.DataFrame(df_test_encoded_missing, columns = df_train_encoded.columns)



In [ ]:
utils.mostrarDatos(df_train_encoded)

Nombre datos: 

--------------------------------------------------------------------------------

Index(['Country_Region_Australia', 'Country_Region_Belgium',
       'Country_Region_Brazil', 'Country_Region_Canada',
       'Country_Region_Chile', 'Country_Region_China',
       'Country_Region_Colombia', 'Country_Region_Denmark',
       'Country_Region_France', 'Country_Region_Germany',
       'Country_Region_India', 'Country_Region_Italy', 'Country_Region_Japan',
       'Country_Region_Malaysia', 'Country_Region_Mexico',
       'Country_Region_Netherlands', 'Country_Region_New Zealand',
       'Country_Region_Pakistan', 'Country_Region_Peru',
       'Country_Region_Russia', 'Country_Region_Spain',
       'Country_Region_Sweden', 'Country_Region_US', 'Country_Region_Ukraine',
       'Country_Region_United Kingdom', 'Lat', 'Long_', 'Confirmed', 'Deaths',
       'Incident_Rate', 'Case_Fatality_Ratio', 'Province_State_encoded',
       'Last_Update_encoded', 'date_encoded'],
      dtype='ob

In [ ]:
%%time
from sklearn.model_selection import train_test_split


def getDataTrain(data, target):

  df_train_encoded_copy =  data.copy()
  Ytrain =  df_train_encoded_copy[target]
  df_train_encoded_copy = df_train_encoded_copy.drop([target], axis=1 )


  ss = StandardScaler()
  ss = ss.fit(df_train_encoded_copy)

  Xtrain0 = ss.transform(df_train_encoded_copy)

  Xtrain = np.nan_to_num(Xtrain0)


  ##测试集
  ss1 = StandardScaler()
  ss1=ss1.fit(Ytrain.values.reshape(-1,1))
  Ytrain=ss1.transform(Ytrain.values.reshape(-1,1))

  return Xtrain, Ytrain, ss1

def getDataTrainEval( data,target, eval_size):
  train, eval = train_test_split(data, test_size=eval_size)
  Xtrain, Ytrain, ssTrain = getDataTrain(train, target)
  Xeval, Yeval, ssEval = getDataTrain(eval, target)
  return Xtrain, Ytrain, ssTrain, Xeval, Yeval , ssEval

def getDataTest(target):

  df_test_encoded_missing_copy =  df_test_encoded_missing.copy()

  Y_DS =   df_test_encoded[target]
  df_test_encoded_missing_copy = df_test_encoded_missing_copy.drop([target], axis=1 )

  ss = StandardScaler()
  ss = ss.fit(df_test_encoded_missing_copy)

  X_DS0 = ss.transform(df_test_encoded_missing_copy)

  X_DS = np.nan_to_num(X_DS0)

  ##测试集
  ss3 = StandardScaler()
  ss3 = ss3.fit(Y_DS.values.reshape(-1,1))
  Y_DS=ss3.transform(Y_DS.values.reshape(-1,1))
  return X_DS, Y_DS, ss3

CPU times: user 0 ns, sys: 10 µs, total: 10 µs
Wall time: 12.6 µs


## 6. Train / test  Model

In [ ]:
%%time

i = 0
eval_size = 0.1
n=5

columnas = df_train_encoded.columns[i:]
try :
  filled_data = pd.read_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Knn_Desarrollo='+str(MODO_DESARROLLO)+'.csv')
except:
  filled_data = pd.DataFrame()
  pass


for column in  columnas:
  if column in filled_data.columns:
    print("Column procesada: ", column)
    i = i+1
    continue
  else:
    print("Column: ", column)
    Xtrain, Ytrain, ss1, Xeval, Yeval , ssEval = getDataTrainEval(df_train_encoded, column, eval_size)

    knn_regresor = KNeighborsRegressor(n, n_jobs=-1)

    knn_regresor.fit(Xtrain, Ytrain)

    Yeval_predict = knn_regresor.predict(Xeval).reshape(-1,1)
    Yeval_predict = ssEval.inverse_transform(Yeval_predict)
    Yeval = ssEval.inverse_transform(Yeval)

    rmse_train = sqrt(metrics.mean_squared_error(Yeval, Yeval_predict))
    print('RMSE training: ', rmse_train)



    print("Termino entrenamiento i=", i)
    print("------------------------------------------------")

    X_DS, Y_DS, ss3 = getDataTest(column)


    Y_DS_predict = knn_regresor.predict(X_DS).reshape(-1,1)
    Y_DS_predict = ss3.inverse_transform(Y_DS_predict)
    Y_DS = ss3.inverse_transform(Y_DS)

    rmse_column =  sqrt(metrics.mean_squared_error(Y_DS, Y_DS_predict))
    print('RMSE column for ', (column),(rmse_column))

    output_list = [item[0] for item in Y_DS_predict]
    filled_data[column] = output_list
    filled_data.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Knn_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)

    i = i+1
    print("Termino test i=", i)
    print("------------------------------------------------")
print("Learning Finished!")



Column procesada:  Country_Region_Australia
Column:  Country_Region_Belgium
RMSE training:  6.351962655115907e-05
Termino entrenamiento i= 1
------------------------------------------------
RMSE column for  Country_Region_Belgium 0.11748136367952268
Termino test i= 2
------------------------------------------------
Column:  Country_Region_Brazil
RMSE training:  0.001978707356695856
Termino entrenamiento i= 2
------------------------------------------------
RMSE column for  Country_Region_Brazil 0.08238735790521472
Termino test i= 3
------------------------------------------------
Column:  Country_Region_Canada
RMSE training:  0.001548213443358062
Termino entrenamiento i= 3
------------------------------------------------
RMSE column for  Country_Region_Canada 0.12107239296864458
Termino test i= 4
------------------------------------------------
Column:  Country_Region_Chile
RMSE training:  0.00042053877593478445
Termino entrenamiento i= 4
-----------------------------------------------

## Guardo el resultados

## Calculo RMSE

In [ ]:
filled_data = pd.read_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Knn_Desarrollo='+str(MODO_DESARROLLO)+'.csv')
filled_data.reset_index(drop=True, inplace=True)
filled_data

,Country_Region_Australia,Country_Region_Belgium,Country_Region_Brazil,Country_Region_Canada,Country_Region_Chile,Country_Region_China,Country_Region_Colombia,Country_Region_Denmark,Country_Region_France,Country_Region_Germany,...,Country_Region_United Kingdom,Lat,Long_,Confirmed,Deaths,Incident_Rate,Case_Fatality_Ratio,Province_State_encoded,Last_Update_encoded,date_encoded
0,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,41.141511,-83.132856,11202.456086,222.649735,22635.353479,1.710664,380.009152,492.664020,280.751915
1,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,38.395165,-94.361276,18278.502176,316.428256,18874.182299,1.404365,363.019680,450.470756,219.964519
2,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,36.320003,-83.528060,32906.235382,483.846309,30060.426413,1.259954,363.019680,469.467723,247.558797
3,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,37.364332,-96.739391,-1594.413505,12.716041,3494.249417,1.148189,154.149112,13.740475,43.801046
4,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,43.383250,-88.242435,17384.330397,201.493626,29173.730363,1.019156,572.889629,223.706956,743.655933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897662,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,40.641070,-90.233921,7372.941379,104.053472,10095.788487,1.713308,123.168310,114.324418,304.946898
897663,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,43.688498,-95.777285,1947.673958,36.313239,8721.480821,1.213591,316.048786,66.532047,190.170697
897664,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,34.295777,-99.274878,-1426.417595,0.714018,22432.637185,0.809289,572.889629,507.461658,310.745696
897665,-0.000012,-0.000033,-0.000008,0.000092,-0.000031,0.000022,-0.000209,-0.000017,-0.000017,-0.000029,...,0.00003,35.265554,-87.798467,1031.199992,23.090671,12831.544563,1.142814,505.931121,154.518001,366.534128


In [ ]:
rmse = utils.rmse(df_test_encoded.to_numpy(), filled_data.to_numpy(), mask_test_open)
print("rmse :", rmse)

rmse : 14276.691255831054


In [ ]:
df_test_encoded, filled_data = utils.obtener_df_imputados(df_test_encoded, filled_data, mask_test_open)

## decodifico

In [ ]:
print("inicio de decoding")

imputed_data_decoded=utils.decode(filled_data, df_train_encoded, PATH_DATA_PROCESADA)

print("imputed_data_decoded.head(5)")
print(imputed_data_decoded.head(5))

print("imputed_data_decoded.to_csv")
imputed_data_decoded.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Knn_full_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)

print("fin de decoding")

inicio de decoding


897667it [01:59, 7515.41it/s]


imputed_data_decoded.head(5)
  Province_State Country_Region          Last_Update        Lat    Long_  \
0          Texas             US  2021-01-02 05:22:33  41.123513 -83.1278   
1          Texas             US  2021-01-02 05:22:33  38.395165 -75.2996   
2          Texas             US  2021-01-02 05:22:33  35.780773 -83.5281   
3          Texas             US  2021-01-02 05:22:33  33.618867 -96.7394   
4          Texas             US  2021-01-02 05:22:33  43.720997 -87.9458   

   Confirmed  Deaths  Incident_Rate  Case_Fatality_Ratio        date  
0      12750     230   23107.035413             1.788235  01-01-2021  
1      10041     167   19549.853001             1.663181  01-01-2021  
2      32906     406   30784.732824             1.325795  01-01-2021  
3        543      27    4167.013225             5.389222  01-01-2021  
4      15026     171   13027.570661             1.138027  01-01-2021  
imputed_data_decoded.to_csv
fin de decoding


In [ ]:
imputed_data_decoded.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Knn_full_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)